# Event Discovery in Long-Horizon Video: Quick Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mesham/event-discovery/blob/main/notebooks/01_demo_quick.ipynb)

This notebook demonstrates physics-inspired event discovery on autonomous driving video.

**Runtime**: ~5 minutes on Colab GPU

## 1. Setup

Install dependencies and clone repository.

In [ ]:
# Install dependencies
!pip install opencv-python numpy matplotlib seaborn tqdm -q

# Clone repository (if not already)
import os
if not os.path.exists('event-discovery'):
    !git clone https://github.com/mesham/event-discovery.git
    %cd event-discovery
else:
    %cd event-discovery

## 2. Download Example Video

We'll use a sample driving video with annotated events.

In [ ]:
# Download example video (placeholder - replace with actual dataset)
!wget -O data/example_video.mp4 https://example.com/driving_video.mp4 -q
!wget -O data/annotations.json https://example.com/annotations.json -q

print("Downloaded example video and annotations")

## 3. Run Hierarchical Energy Method

Our main physics-inspired approach.

In [ ]:
import sys
sys.path.append('src')

from methods.hierarchical_energy import HierarchicalEnergyMethod, EnergyConfig
from core.video_processor import visualize_detections

# Configure method
config = EnergyConfig(
    weight_motion=0.3,
    weight_interaction=0.3,
    weight_scene_change=0.2,
    weight_uncertainty=0.2,
    thresholds=[2.0, 1.5, 1.0],
    top_k=10
)

# Initialize method
method = HierarchicalEnergyMethod(config)

# Process video
print("Processing video with Hierarchical Energy method...")
detected_events = method.process_video('data/example_video.mp4')

print(f"\nDetected {len(detected_events)} events:")
for i, event in enumerate(detected_events):
    print(f"  Event {i+1}: {event.start_time:.2f}s - {event.end_time:.2f}s")

## 4. Visualize Results

Create annotated video with detected events highlighted.

In [ ]:
import json

# Load ground truth annotations
with open('data/annotations.json', 'r') as f:
    annotations = json.load(f)

# Create visualization
print("Creating visualization video...")
visualize_detections(
    'data/example_video.mp4',
    detected_events,
    'results/videos/hierarchical_energy_demo.mp4',
    annotations['events']
)

print("Visualization complete!")

## 5. Display Results

Show detected events in notebook.

In [ ]:
from IPython.display import Video

# Display output video
Video('results/videos/hierarchical_energy_demo.mp4', width=800)

## 6. Compute Metrics

Evaluate precision and recall.

In [ ]:
def compute_metrics(detected, ground_truth, iou_threshold=0.5):
    """Compute precision, recall, and F1 score."""
    
    def iou(window1, window2):
        """Intersection over union for temporal windows."""
        start = max(window1['start_time'], window2.start_time)
        end = min(window1['end_time'], window2.end_time)
        
        if start >= end:
            return 0.0
        
        intersection = end - start
        union = (window1['end_time'] - window1['start_time'] + 
                 window2.end_time - window2.start_time - intersection)
        
        return intersection / union
    
    # Match detections to ground truth
    tp = 0
    matched_gt = set()
    
    for det in detected:
        best_iou = 0
        best_gt_idx = None
        
        for i, gt in enumerate(ground_truth):
            if i not in matched_gt:
                overlap = iou(gt, det)
                if overlap > best_iou:
                    best_iou = overlap
                    best_gt_idx = i
        
        if best_iou >= iou_threshold:
            tp += 1
            matched_gt.add(best_gt_idx)
    
    fp = len(detected) - tp
    fn = len(ground_truth) - tp
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tp': tp,
        'fp': fp,
        'fn': fn
    }

# Compute metrics
metrics = compute_metrics(detected_events, annotations['events'])

print("\nEvaluation Metrics:")
print(f"  Precision: {metrics['precision']:.3f}")
print(f"  Recall:    {metrics['recall']:.3f}")
print(f"  F1 Score:  {metrics['f1']:.3f}")
print(f"\n  True Positives:  {metrics['tp']}")
print(f"  False Positives: {metrics['fp']}")
print(f"  False Negatives: {metrics['fn']}")

## 7. Compare Energy Components

Visualize contribution of each energy term.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Re-extract features for analysis
all_windows = method.processor.chunk_video('data/example_video.mp4')
all_features = method.extract_features(all_windows, level=2)

# Extract feature arrays
motion = [f['motion'] for f in all_features]
interaction = [f['interaction'] for f in all_features]
scene_change = [f['scene_change'] for f in all_features]
uncertainty = [f['uncertainty'] for f in all_features]

# Compute total energy
total_energy = method.compute_energy(all_features)

# Create stacked area plot
fig, ax = plt.subplots(figsize=(14, 6))

time_points = [w.start_time for w in all_windows]

ax.fill_between(time_points, 0, motion, alpha=0.5, label='Motion')
ax.fill_between(time_points, motion, np.array(motion) + np.array(interaction), 
                alpha=0.5, label='Interaction')
ax.plot(time_points, total_energy, 'k-', linewidth=2, label='Total Energy')

# Mark detected events
for event in detected_events:
    ax.axvspan(event.start_time, event.end_time, alpha=0.3, color='red', label='Detected')

# Mark ground truth
for gt in annotations['events']:
    ax.axvspan(gt['start_time'], gt['end_time'], alpha=0.2, color='blue', 
              linestyle='--', label='Ground Truth')

ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Energy', fontsize=12)
ax.set_title('Event Energy Over Time', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/energy_timeline.pdf', dpi=300, bbox_inches='tight')
plt.show()

## 8. Next Steps

- Try different configuration parameters
- Compare with other methods (see `02_full_comparison.ipynb`)
- Run on your own videos
- Tune weights for your specific domain

**Full documentation**: [README.md](../README.md)